In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetMobile,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetMobile(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 1056)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ...........

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s


[CV] ........................... C=0.01, score=0.928753, total=   0.7s
[CV] C=10.0 ..........................................................
[CV] C=100.0 .........................................................
[CV] ........................... C=0.01, score=0.931298, total=   0.9s
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.945362, total=   3.5s
[CV] C=1000.0 ........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    6.2s


[CV] .......................... C=100.0, score=0.931385, total=   5.3s
[CV] C=100000.0 ......................................................
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.921120, total=   5.6s
[CV] C=1000000.0 .....................................................
[CV] ......................... C=1000.0, score=0.940204, total=   5.0s
[CV] ............................ C=1.0, score=0.940280, total=   6.1s
[CV] ............................ C=1.0, score=0.947837, total=   6.1s
[CV] ........................... C=10.0, score=0.931385, total=   5.8s
[CV] ........................... C=10.0, score=0.942748, total=   5.8s
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.935115, total=   5.7s
[CV] C=1000000.0 .....................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.923664, total=   5.1s
[CV] .................... C=100000000.0, score=0.931298, total=   5.1s
[CV] ................... C=1000000000.0, score=0.936387, total=   4.4s
[CV] ................... C=1000000000.0, score=0.932656, total=   5.1s
[CV] ................... C=1000000000.0, score=0.937659, total=   4.2s
[CV] .................. C=10000000000.0, score=0.932656, total=   3.4s
[CV] .................. C=10000000000.0, score=0.923664, total=   3.0s
[CV] .................. C=10000000000.0, score=0.931298, total=   2.8s
[CV] ................... C=1000000000.0, score=0.923664, total=   3.8s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   15.2s remaining:    1.3s


[CV] .................. C=10000000000.0, score=0.936387, total=   3.5s
[CV] ................... C=1000000000.0, score=0.933842, total=   3.9s
[CV] .................. C=10000000000.0, score=0.941476, total=   3.7s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   15.8s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [10]:
top_model.cv_results_

{'mean_fit_time': array([0.61364388, 3.69968224, 5.5827147 , 5.6175921 , 5.47359724,
        5.18078246, 5.17058177, 5.40970001, 5.3948483 , 5.35076661,
        5.29702039, 4.27500634, 3.27567744]),
 'mean_score_time': array([0.03516564, 0.04167719, 0.06213131, 0.07921047, 0.05899029,
        0.07704668, 0.01911473, 0.09311128, 0.08895669, 0.08296127,
        0.06163716, 0.03365798, 0.0266665 ]),
 'mean_test_score': array([0.9374205 , 0.94301704, 0.93996439, 0.93233274, 0.93284152,
        0.93258713, 0.93156958, 0.93156958, 0.9310608 , 0.93156958,
        0.93207835, 0.93284152, 0.9330959 ]),
 'mean_train_score': array([0.96336818, 0.98963359, 0.99961842, 1.        , 0.99993641,
        0.99993641, 0.99993641, 0.99987281, 0.99987281, 0.99993641,
        0.99987281, 0.99993641, 1.        ]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    10000000000.0],
    

In [11]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93742, std: 0.00730, params: {'C': 0.01},
 mean: 0.94302, std: 0.00868, params: {'C': 0.1},
 mean: 0.93996, std: 0.00765, params: {'C': 1.0},
 mean: 0.93233, std: 0.00698, params: {'C': 10.0},
 mean: 0.93284, std: 0.00593, params: {'C': 100.0},
 mean: 0.93259, std: 0.00580, params: {'C': 1000.0},
 mean: 0.93157, std: 0.00599, params: {'C': 10000.0},
 mean: 0.93157, std: 0.00518, params: {'C': 100000.0},
 mean: 0.93106, std: 0.00530, params: {'C': 1000000.0},
 mean: 0.93157, std: 0.00599, params: {'C': 10000000.0},
 mean: 0.93208, std: 0.00474, params: {'C': 100000000.0},
 mean: 0.93284, std: 0.00492, params: {'C': 1000000000.0},
 mean: 0.93310, std: 0.00589, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9430
